<a href="https://colab.research.google.com/github/anutkk/llms/blob/master/Tanchuma_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tanchuma - char-level tokenization

In [1]:
# !pip install --progress-bar off torch numpy transformers datasets tiktoken wandb tqdm

In [2]:
# !git clone https://github.com/karpathy/nanoGPT.git
# !cp nanoGPT/model.py .

In [3]:
# try:
#   import google.colab
#   IN_COLAB = True
# except:
#   IN_COLAB = False
# #for colab only
# if IN_COLAB:
#   !git clone https://github.com/anutkk/llms.git
#   !cp llms/tanchuma_text.py .
#   !cp -r llms/tanchuma_text .

Download and tokenize data:

In [4]:
"""
Prepare the Shakespeare dataset for character-level language modeling.
So instead of encoding with GPT-2 BPE tokens, we just map characters to ints.
Will save train.bin, val.bin containing the ids, and meta.pkl containing the
encoder and decoder and some other related info.
"""
import os
import pickle
import requests
import numpy as np
import tanchuma_text

train_data, test_data = tanchuma_text.get_data()

data = "\n".join(train_data) + "\n".join(test_data)

print(f"length of dataset in characters: {len(data):,}")

# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# create the train and test splits
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode both to integers
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile('train.bin')
val_ids.tofile('val.bin')

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open('meta.pkl', 'wb') as f:
    pickle.dump(meta, f)

# length of dataset in characters: 3,156,907
# all the unique characters:
#  "'אבגדהוזחטיךכלםמןנסעףפץצקרשת
# vocab size: 31
# train has 2,841,216 tokens
# val has 315,691 tokens

length of dataset in characters: 3,156,907
all the unique characters: 
 "'אבגדהוזחטיךכלםמןנסעףפץצקרשת
vocab size: 31
train has 2,841,216 tokens
val has 315,691 tokens


Define model architecture:

In [5]:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False


dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 10
n_head = 8
n_embd = 384*2
dropout = 0.2

grad_clip = 1.0

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 6000
lr_decay_iters = 6000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta1 = 0.9
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

weight_decay = 1e-1
warmup_iters = 100 # not super necessary potentially

Train:

In [6]:
import os
import time
import math
import pickle
from contextlib import nullcontext

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

# various inits, derived attributes, I/O setup
master_process = True
seed_offset = 0
ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader
data_dir = os.path.join('data', dataset)
train_data = np.memmap('train.bin', dtype=np.uint16, mode='r')
val_data = np.memmap('val.bin', dtype=np.uint16, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=False, vocab_size=None, dropout=dropout) # start with model_args from command line

# init a new model from scratch
print("Initializing a new model from scratch")
# determine the vocab size we'll use for from-scratch training
if meta_vocab_size is None:
    print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
gptconf = GPTConfig(**model_args)
model = GPT(gptconf)

# crop down the model block size if desired, using model surgery
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size # so that the checkpoint will have the right value
model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
checkpoint = None # free up memory

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0


# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)


# training loop
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model # unwrap DDP container if needed
running_mfu = -1.0
while True:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))


    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        with ctx:
            logits, loss = model(X, Y)
            loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        X, Y = get_batch('train')
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

tokens per iteration will be: 16,384
Initializing a new model from scratch
defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)
number of parameters: 109.43M
num decayed parameter tensors: 42, with 109,608,960 parameters
num non-decayed parameter tensors: 21, with 16,128 parameters
using fused AdamW: True
compiling the model... (takes a ~minute)
step 0: train loss 10.8580, val loss 10.8614


[2023-09-17 21:23:40,484] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-09-17 21:23:40,804] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-09-17 21:23:41,488] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-09-17 21:23:41,783] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-09-17 21:23:42,237] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-09-17 21:23:42,533] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-09-17 21:23:42,978] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-09-17 21:23:43,271] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-09-17 21:23:43,712] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-09-17 21:23:4

iter 0: loss 10.8786, time 29749.26ms, mfu -100.00%
iter 10: loss 6.4341, time 104.46ms, mfu 34.19%
iter 20: loss 4.8352, time 105.40ms, mfu 34.16%
iter 30: loss 2.9700, time 104.89ms, mfu 34.15%
iter 40: loss 2.6100, time 104.47ms, mfu 34.16%
iter 50: loss 2.5579, time 104.24ms, mfu 34.17%
iter 60: loss 2.5304, time 104.42ms, mfu 34.17%
iter 70: loss 2.5314, time 104.89ms, mfu 34.16%
iter 80: loss 2.5223, time 104.57ms, mfu 34.16%
iter 90: loss 2.5048, time 104.46ms, mfu 34.16%
iter 100: loss 2.5229, time 105.03ms, mfu 34.15%
iter 110: loss 2.4891, time 105.14ms, mfu 34.13%
iter 120: loss 2.4589, time 104.95ms, mfu 34.12%
iter 130: loss 2.4594, time 104.69ms, mfu 34.12%
iter 140: loss 2.4318, time 105.32ms, mfu 34.10%
iter 150: loss 2.4132, time 105.42ms, mfu 34.08%
iter 160: loss 2.3339, time 105.26ms, mfu 34.06%
iter 170: loss 2.3019, time 104.87ms, mfu 34.06%
iter 180: loss 2.2263, time 104.73ms, mfu 34.07%
iter 190: loss 2.2196, time 104.70ms, mfu 34.07%
iter 200: loss 2.1972, tim

Sample inference:

In [7]:
# import os
# import pickle
# from contextlib import nullcontext
# import torch
import tiktoken

# -----------------------------------------------------------------------------
num_samples = 5 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# model
# init from a model saved in a specific directory
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)


model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

meta_path = 'meta.pkl'
print(f"Loading meta from {meta_path}...")
with open(meta_path, 'rb') as f:
    meta = pickle.load(f)
stoi, itos = meta['stoi'], meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])



# prompt
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

# run generation
with torch.inference_mode():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')

number of parameters: 109.43M
Loading meta from meta.pkl...

יום שנאמר ושם המפורש הגדול אשר נתן להם ולא נתן להם קלון אלא לפיכך כתיב ואשמיד פריו ממעל ושרשיו מתחת כסאו שרשיו מתחת כסא הכבוד שנאמר ויעמד פרח ויפריו מתחת כנפיו וישב ישראל וגו' ויחנו בחרט למה שנו רבותינו למה שהיתה המילה שמתחת כנפיו לו אותה שעה לא חטא ויאמר ה' אל משה נטה ידך על המטה והרבה רגליך את הסכנה לו כשישראל כותשין את הסכנה ונופלין במטה ומה כתיב שם ויבאו שני המלאכים סדומה ויחנו לפני המלאכים הקב"ה אמר ר' אלעזר ברבי שמעון בן יוחי אומר מאזני ומה היה עם לבבך היה עמלק אמר ר' יוסי בר חנינא אם יהיה
---------------

בושה זו רשות לאחרים ויבאו עלינו כי אני ה' אלהיך וגו' אמרו לו הרי עתיד הקב"ה לעשות את הדין להם את הדין וכיון שהוא אומר עד עכשיו אין אנו יודעין שאין הרי מתים שאין לנו כלום אלא במידה שאמרו נהיה ראשית תחת הנשים וגו' וזה הדבר כתוב אחד אומר ויודע ה' אל משה כתב לך את הדבר אשר דברת לאמר בהעלתך את הדברים האלה כתב לך את הדברים האלה מה כתב אחריו והודעתם לבניך ובניך למשה ומה אמר להם הקב"ה אתם מבקשים להקריב לפני לא כן אלא קרבנות ש